<a href="https://colab.research.google.com/github/nikhilgunti/Skin-Cancer-Detection/blob/master/DLPY_CNN_Skin_cancer_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Author: Nilesh Jakhotiya
#Email:  jakhotia.nilesh@gmail.com
from swat import CAS
import swat
from swat.cas.table import merge
import os
import dlpy
import numpy
import matplotlib.image as mping
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd

from dlpy.images import ImageTable
from dlpy.applications import UNet
from dlpy.model import Optimizer, Model, DataSpec, MomentumSolver, AdamSolver, StepLR
from dlpy.layers import *
from dlpy.utils import display_segmentation_images, create_segmentation_table, display_segmentation_results
from dlpy.sequential import Sequential

In [0]:
cashost='rdcgrd001'
casport=37645
casauth='C:/Users/nijakh/_authinfo'

# Start CAS session
conn = CAS(cashost, casport, authinfo=casauth, caslib="casuser")
conn.loadactionset('image')
conn.loadactionset('deeplearn')

NOTE: Added action set 'image'.
NOTE: Added action set 'deeplearn'.


[actionset]

 'deeplearn'

+ Elapsed: 0.0669s, user: 0.273s, sys: 0.682s, mem: 30.9mb

In [0]:
# load_data
image_data=conn.loadTable( caslib="casuser",path="hmnist_28_28_L.csv", importoptions={"filetype":"csv"},casout={"name":"hmnist"})


NOTE: Cloud Analytic Services made the file hmnist_28_28_L.csv available as table HMNIST in caslib CASUSERHDFS(nijakh).


In [0]:
#Split data into train and test
conn.loadactionset('sampling')
conn.sampling.stratified(table="hmnist",target='label',samppct=30,samppct2=10,output={"casout":{"name":"hmnist_sample","replace":True},"copyvars":"ALL"})

NOTE: Added action set 'sampling'.
NOTE: The PARTIND option is set to be TRUE for segment stratified sampling.
NOTE: Stratified sampling is in effect.
NOTE: Using SEED=383246538 for sampling.


[OutputCasTables]

                 casLib           Name Label   Rows  Columns  \
 0  CASUSERHDFS(nijakh)  hmnist_sample        10015      786   
 
                                             casTable  
 0  CASTable('hmnist_sample', caslib='CASUSERHDFS(...  

[PartIndMap]

 Partition Indicator Map
 
   PartIndName TargetName
 0  _PartInd1_      label

[STRAFreq]

 Frequencies
 
   TargetName TargetLevel   NObs  NSamp1  NSamp2
 0      label              10015    3004    1002

+ Elapsed: 4.05s, user: 14.9s, sys: 19.9s, mem: 1.15e+04mb

In [0]:
#split into Train, Validation and Test sets
conn.loadactionset('table')
conn.table.save(caslib="casuser",name="hmnist_train",replace=True,table={"name":'hmnist_sample',"where":"_partind1_=0" })
conn.table.save(caslib="casuser",name="hmnist_valid",replace=True,table={"name":'hmnist_sample',"where":"_partind1_=1" })
conn.table.save(caslib="casuser",name="hmnist_test",replace=True,table={"name":'hmnist_sample',"where":"_partind1_=2" })

NOTE: Added action set 'table'.
NOTE: Cloud Analytic Services saved the file hmnist_train.sashdat in caslib CASUSER(nijakh).
NOTE: Cloud Analytic Services saved the file hmnist_valid.sashdat in caslib CASUSER(nijakh).
NOTE: Cloud Analytic Services saved the file hmnist_test.sashdat in caslib CASUSER(nijakh).


[caslib]

 'CASUSER(nijakh)'

[name]

 'hmnist_test.sashdat'

+ Elapsed: 0.349s, user: 1.62s, sys: 1.18s, mem: 2.78e+03mb

In [0]:
# load_data
image_data=conn.loadTable( caslib="casuser",path="hmnist_train.sashdat", importoptions={"filetype":"hdat"},casout={"name":"train","replace":True})
image_data=conn.loadTable( caslib="casuser",path="hmnist_valid.sashdat", importoptions={"filetype":"hdat"},casout={"name":"valid","replace":True})
image_data=conn.loadTable( caslib="casuser",path="hmnist_test.sashdat", importoptions={"filetype":"hdat"},casout={"name":"test","replace":True})


NOTE: Cloud Analytic Services made the file hmnist_train.sashdat available as table TRAIN in caslib CASUSERHDFS(nijakh).
NOTE: Cloud Analytic Services made the file hmnist_valid.sashdat available as table VALID in caslib CASUSERHDFS(nijakh).
NOTE: Cloud Analytic Services made the file hmnist_test.sashdat available as table TEST in caslib CASUSERHDFS(nijakh).


In [0]:
conn.table.tableInfo(name="TEST")      
#conn.table.tableDetails(name="test")

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,TEST,1002,786,0,utf-8,2020-04-04T13:17:33-04:00,2020-04-04T13:17:33-04:00,2020-04-04T13:17:33-04:00,UTF8,1.901640e+09,...,0,0,0,hmnist_test.sashdat,CASUSER(nijakh),0,nijakh,,2020-04-04T13:17:32-04:00,1.901640e+09


In [0]:
#define empty cnn model
conn.deepLearn.buildModel(
modelTable={"name":"hmnistcnn", "replace":True},
type="CNN")

conn.deepLearn.modelInfo(                        
modelTable={"name":"hmnistcnn"}
)
  

,Descr,Value
0,Model Name,hmnistcnn
1,Model Type,Convolutional Neural Network
2,Number of Layers,0
3,Number of Input Layers,0
4,Number of Output Layers,0
5,Number of Convolutional Layers,0
6,Number of Pooling Layers,0
7,Number of Fully Connected Layers,0


In [0]:
#Add layers to empty cnn model

#Add input layer
conn.deepLearn.addLayer(          
layer={"type":"INPUT","height":28,
"width":28},    
modelTable={"name":"hmnistcnn"} ,   
name="data")           

#Conv layer with 32 filters of 3 by 3
conn.deepLearn.addLayer(
layer={"type":"convo",
"act":"relu",
"nfilters":32,
"height":3,
"width":3,
"dropout":.2    
},      
modelTable={"name":"hmnistcnn"} ,  
name="conv1",
srcLayers={"data"})   

#batch normalization layer 
conn.deepLearn.addLayer(
layer={"type":"batchnorm"} ,     
modelTable={"name":"hmnistcnn"} ,  
name="batch1",
srcLayers={"conv1"})  

#pooling layer 
conn.deepLearn.addLayer(
layer={"type":"pool","dropout":.2,"height":3, "width":3}  ,    
modelTable={"name":"hmnistcnn"}   ,
name="pool1",
srcLayers={"batch1"})

#Conv layer with 64 filters of 4 by 4
conn.deepLearn.addLayer(
layer={"type":"convo",
"act":"relu",
"nfilters":64,
"height":4,
"width":4,
"dropout":.2,    
}      ,
modelTable={"name":"hmnistcnn"},   
name="conv2",
srcLayers={"pool1"})   

#batch normalization layer 
conn.deepLearn.addLayer(
layer={"type":"batchnorm"}  ,    
modelTable={"name":"hmnistcnn"} ,  
name="batch2",
srcLayers={"conv2"})  

#pooling layer 
conn.deepLearn.addLayer(
layer={"type":"pool","dropout":.2,"height":3, "width":3}   ,   
modelTable={"name":"hmnistcnn"}  , 
name="pool2",
srcLayers={"batch2"})

#Conv layer with 128 filters of 3 by 3
conn.deepLearn.addLayer(
layer={"type":"convo",
"act":"relu",
"nfilters":128,
"height":3,
"width":3,
"dropout":.2,    
}      ,
modelTable={"name":"hmnistcnn"},   
name="conv3",
srcLayers={"pool2"})   

#batch normalization layer 
conn.deepLearn.addLayer(
layer={"type":"batchnorm"}  ,    
modelTable={"name":"hmnistcnn"} ,  
name="batch3",
srcLayers={"conv3"})  

#pooling layer 
conn.deepLearn.addLayer(
layer={"type":"pool","dropout":.2,"height":3, "width":3}   ,   
modelTable={"name":"hmnistcnn"}  , 
name="pool3",
srcLayers={"batch3"})

#output layer
conn.deepLearn.addLayer(
layer={"type":"output",
"act":'softmax',
"fullConnect":True
},    
modelTable={"name":"hmnistcnn"}  ,   
name="outlayer",
srcLayers={"pool3"})           


,casLib,Name,Rows,Columns,casTable
0,CASUSERHDFS(nijakh),hmnistcnn,134,5,"CASTable('hmnistcnn', caslib='CASUSERHDFS(nija..."


In [0]:
conn.deepLearn.modelInfo(                        
modelTable={"name":"hmnistcnn"}
)

,Descr,Value
0,Model Name,hmnistcnn
1,Model Type,Convolutional Neural Network
2,Number of Layers,11
3,Number of Input Layers,1
4,Number of Output Layers,1
5,Number of Convolutional Layers,3
6,Number of Pooling Layers,3
7,Number of Fully Connected Layers,0
8,Number of Batch Normalization Layers,3


In [0]:
#create input variable list 
vars=[]
for x in range(784):
    tmpx=str(x).zfill(4)
    vname="pixel{0}".format(tmpx)
    vars.append(vname)
    
#vars     

In [0]:
#Train the model
conn.deepLearn.dlTrain(
optimizer={"algorithm":{"method":"ADAM"}, "minibatchsize":90, "maxepochs":100},
inputs=vars,
modelTable={"name":"hmnistcnn"},
modelWeights={"name":"hmnistTrainedWeights",
"replace":True},
#forceequalpadding=True,
table={"name":"train"},
nominals=[{"name":"label"}],
target="label",
validtable={"name":"valid"})


NOTE: Due to data distribution, miniBatchSize has been limited to 83.


,Descr,Value
0,Model Name,hmnistcnn
1,Model Type,Convolutional Neural Network
2,Number of Layers,11
3,Number of Input Layers,1
4,Number of Output Layers,1
5,Number of Convolutional Layers,3
6,Number of Pooling Layers,3
7,Number of Fully Connected Layers,0
8,Number of Batch Normalization Layers,3
9,Number of Weight Parameters,110368


In [0]:
conn.deepLearn.dlTune(inputs=vars,
modelTable={"name":"hmnistcnn"},
initweights={"name":"hmnistTrainedWeights"},
modelWeights={"name":"hmnistTrainedWeights2","replace":True},
#forceequalpadding=True,
table={"name":"train"},
nominals=[{"name":"label"}],
target="label",
validtable={"name":"valid"},
optimizer={"algorithm":"ADAM", "dropout":{"lowerBound":.1, "upperBound":.5},"minibatchsize":{"lowerBound":30, "upperBound":90}, "maxepochs":200 } )

NOTE: Due to data distribution, miniBatchSize has been limited to 83.


,Descr,Value
0,Model Name,hmnistcnn
1,Model Type,Convolutional Neural Network
2,Number of Layers,11
3,Number of Input Layers,1
4,Number of Output Layers,1
5,Number of Convolutional Layers,3
6,Number of Pooling Layers,3
7,Number of Fully Connected Layers,0
8,Number of Batch Normalization Layers,3
9,Number of Weight Parameters,110368


In [0]:
conn.deepLearn.dlScore(                
      casOut={"name":"test_scored",
              "replace":True},copyvars={"label"},
      initWeights={"name":"hmnistTrainedWeights2"},
      modelTable={"name":"hmnistcnn"},
      table={"name":"test"}
)

[OutputCasTables]

                 casLib         Name  Rows  Columns  \
 0  CASUSERHDFS(nijakh)  test_scored  1002       11   
 
                                             casTable  
 0  CASTable('test_scored', caslib='CASUSERHDFS(ni...  

[ScoreInfo]

                          Descr         Value
 0  Number of Observations Read          1002
 1  Number of Observations Used          1002
 2  Misclassification Error (%)      29.34132
 3                   Loss Error      0.874949

+ Elapsed: 1.47s, user: 14s, sys: 44.4s, mem: 2.38e+04mb

In [0]:
#test_out=conn.CASTable('test_scored')
#test_out.fetch(maxrows=100)
conn.table.fetch(table="test_scored",maxrows=100,to=1000)

,label,_DL_P0_,_DL_P1_,_DL_P2_,_DL_P3_,_DL_P4_,_DL_P5_,_DL_P6_,_DL_PredName_,_DL_PredP_,_DL_PredLevel_
0,4.0,0.037584,0.929015,0.012932,0.000143,0.000012,0.000077,0.020238,4,0.929015,1.0
1,4.0,0.013219,0.875665,0.019800,0.000557,0.000199,0.001039,0.089521,4,0.875665,1.0
2,4.0,0.015372,0.933273,0.024345,0.000040,0.000018,0.000768,0.026183,4,0.933273,1.0
3,4.0,0.114962,0.815911,0.023468,0.001147,0.000135,0.001333,0.043044,4,0.815911,1.0
4,4.0,0.018139,0.950554,0.015975,0.000058,0.000022,0.000351,0.014900,4,0.950554,1.0
...,...,...,...,...,...,...,...,...,...,...,...
95,6.0,0.068206,0.468080,0.291563,0.013394,0.001625,0.036614,0.120517,4,0.468080,1.0
96,6.0,0.022131,0.372075,0.196481,0.014348,0.009550,0.002545,0.382870,2,0.382870,6.0
97,6.0,0.002769,0.661510,0.298245,0.000206,0.000052,0.000050,0.037168,4,0.661510,1.0
98,6.0,0.013431,0.097461,0.665291,0.007954,0.007284,0.000411,0.208167,6,0.665291,2.0


In [0]:
conn.loadactionset('freqtab')
conn.freqtab.freqtab(table="test_scored",tabulate=[{"vars":{"label"},"cross":{"_DL_PredLevel_"}}])

NOTE: Added action set 'freqtab'.


[LevelInfo]

 Variable Level Information
 
          Variable                                     Label  Levels  \
 0  _DL_PredLevel_  Predicted level with highest probability       5   
 1           label                                                 7   
 
                                               Values  
 0             1            2            3        ...  
 1                                      0 1 2 3 4 5 6  

[NObs]

 Number of Observations
 
       RowId                  Description   Value
 0     NREAD  Number of Observations Read  1002.0
 1     NUSED  Number of Observations Used  1002.0
 2  NWEIGHTS               Sum of Weights  1002.0

[Table1.CrossList]

 Table of label by _DL_PredLevel_
 
    F_label  label F__DL_PredLevel_  _DL_PredLevel_  Frequency     Percent  \
 0        0    0.0                1             1.0        2.0    0.199601   
 1        0    0.0                2             2.0        4.0    0.399202   
 2        0    0.0                4             4.0        1.0    0.099800   
 3        0    0.0                6             6.0       25.0    2.495010   
 4        0    0.0            Total             NaN       32.0    3.193613   
 5        1    1.0                1             1.0       19.0    1.896208   
 6        1    1.0                2             2.0        5.0    0.499002   
 7        1    1.0                3             3.0        1.0    0.099800   
 8        1    1.0                6             6.0       27.0    2.694611   
 9        1    1.0            Total             NaN       52.0    5.189621   
 10       2    2.0                1             1.0       59.0    5.888224   
 11       2    2.0                2             2.0       13.0    1.297405   
 12       2    2.0                4             4.0        1.0    0.099800   
 13       2    2.0                6             6.0       39.0    3.892216   
 14       2    2.0            Total             NaN      112.0   11.177645   
 15       3    3.0                1             1.0        7.0    0.698603   
 16       3    3.0                2             2.0        1.0    0.099800   
 17       3    3.0                6             6.0        2.0    0.199601   
 18       3    3.0            Total             NaN       10.0    0.998004   
 19       4    4.0                1             1.0      636.0   63.473054   
 20       4    4.0                2             2.0       15.0    1.497006   
 21       4    4.0                6             6.0       20.0    1.996008   
 22       4    4.0            Total             NaN      671.0   66.966068   
 23       5    5.0                1             1.0       11.0    1.097804   
 24       5    5.0                2             2.0        1.0    0.099800   
 25       5    5.0                6             6.0        1.0    0.099800   
 26       5    5.0            Total             NaN       13.0    1.297405   
 27       6    6.0                1             1.0       63.0    6.287425   
 28       6    6.0                2             2.0       31.0    3.093812   
 29       6    6.0                6             6.0       18.0    1.796407   
 30       6    6.0            Total             NaN      112.0   11.177645   
 31   Total    NaN                1             1.0      797.0   79.540918   
 32   Total    NaN                2             2.0       70.0    6.986028   
 33   Total    NaN                3             3.0        1.0    0.099800   
 34   Total    NaN                4             4.0        2.0    0.199601   
 35   Total    NaN                6             6.0      132.0   13.173653   
 36   Total    NaN            Total             NaN     1002.0  100.000000   
 
     RowPercent  ColPercent  
 0     6.250000    0.250941  
 1    12.500000    5.714286  
 2     3.125000   50.000000  
 3    78.125000   18.939394  
 4   100.000000         NaN  
 5    36.538462    2.383940  
 6     9.615385    7.142857  
 7     1.923077  100.000000  
 8    51.923077   20.454545  
 9   100.0000